In [5]:
!pip install dill
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=786bbfa5b6a03fad9e3849f8c00a65c51a042729c376448bed53efdd8759a790
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.svm import SVC, LinearSVC
import joblib
import dill
from lime.lime_tabular import LimeTabularExplainer
from datetime import datetime
%run '/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/gral_functions/gral_funcs_box_data.ipynb'
%run '/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/1_new_models_since_raw_data/funcs_model_2_short.ipynb'
%run '/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/gral_functions/pipeline_cleaner_raw_data.ipynb'
%xmode #verbose
import warnings
warnings.filterwarnings('ignore')

In [7]:
# cluster = joblib.load('/content/cluster_object_kmeans_4_clus.pkl')
# scaler = joblib.load('/content/scaler_standard.pkl')
# model = joblib.load('/content/model_object_svc_.pkl')
# with open('/content/lime_explainer_svc.dill', 'rb') as file:
#   explainer = dill.load(file)

In [10]:
# pd.set_option('display.max_columns', 50)
# # lectura de la data nueva
# bo = pd.read_csv('/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/__data__/boxers_info_.csv')
# fi = pd.read_csv('/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/__data__/fight_info_real.csv')
# fr = pd.read_csv('/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/__data__/fight_results_real.csv')
# # merge and split new data
# ndf, new_id_df, new_other_cols = box_data_merger_spliter(bo,fi,fr)

# # separate ndf to get the columns with res nan values
# trial = ndf[ndf.result.isna()]

Make predictions

In [56]:
class Model_predictions:
  '''
  Class to make predictions to first df received from box_data_merger_spliter() function.
  Warning:
    It should be runned data_column_sep() and new_preds() functions.
  '''
  def __init__(self, cluster_obj, scaler_obj, model_obj, explainer):
    '''
    Constructor method.
    Parameters:
      cluster_obj: Cluster object fitted.
      scaler_obj: Scaler object fitted.
      model_obj: Model object fitted.
      explainer: Lime tabular explainer fitted.
    '''
    self.model = model_obj
    self.cluster = cluster_obj
    self.scaler = scaler_obj
    self.explainer = explainer

  def predict(self, data, id_df):
    res_new, pelea_new, X_new, y_new = data_column_sep(data) # real data
    xn_clus, xn_fe, xn_scal, yn_pred = new_preds(X_new, self.cluster, self.scaler, self.model)
    df1, df2, df3 = check_predictions(res_new, pelea_new, yn_pred, id_df, xn_clus)
    return df3, xn_scal

  def confidence_check(self, data):
    df = data.copy()
    mes1_list = []
    mes2_list = []
    for i, e in zip(df.cluster, df.prob_win):
      if i == 0:
        mes2 = 'fail 19 % of 1150 (val, test)'
        if (e > .35) & (e < .45):
          mes1 = 'warning: .35 to .45, fail 30 %'
        elif (e > .45) & (e < .65):
          mes1 = 'WARNING: .45 to .65, fail 50 %'
        elif (e > .15) & (e < .35):
          mes1 = 'WARNING: .15 to .35, fail 50 %'
        else:
          mes1 = ''
      if i == 1:
        mes2 = 'fail 4 % of 90 (warning: few data)(val, test)'
        if (e > .25) & (e < .35):
          mes1 = 'WARNING: .25 to .35, fail 60 %'
        else:
          mes1 = ''
      if i == 2:
        mes2 = 'not fail, WARNING: only 25 (val, test)'
        mes1 = ''
      if i == 3:
        mes2 = 'fail 10 % of 285 (warning: few data)(val, test)'
        if (e > .45) & (e < .56):
          mes1 = 'warning: .45 to .56, fail 38 %'
        elif (e > .25) & (e < .45):
          mes1 = 'WARNING: .25 to .45, fail 50 %'
      mes2_list.append(mes2)
      mes1_list.append(mes1)
    df['data_cuant'] = mes2_list
    df['confidence'] = mes1_list
    return df

  def select_columns(self, data):
    df = data.copy()
    df = df[['initial_index', 'fight_id', 'result', 'endround', 'endtype',
            'cluster', 'boxer1_pred', 'goodpred', 'prob_win', 'data_cuant', 'confidence',
            'boxer1', 'boxer2', 'dif_bet', 'dif_len_text',
            'dif_edad', 'dif_height', 'dif_w', 'dif_wk_perc',
            'dif_l', 'dif_lk_perc', 'modelo', 'b1_id', 'b2_id']]
    return df

  def decision_table(self, data):
    df = data.copy()
    df['dif_edad'] = [f'{i} - {e} = {i - e}' for i,e in zip(df.birth_b1, df.birth_b2)]
    df['dif_height'] = [f'{i} - {e} = {i - e}' for i,e in zip(df.height_b1, df.height_b2)]
    df['dif_w'] = [f'{i} - {e} = {i - e}' for i,e in zip(df.b1_w, df.b2_w)]
    df['dif_wk_perc'] = [f'{round(i/ie, 2)} - {round(e/ei,2)} = {round((i/ie) - (e/ei), 2)}'\
                              if (ie > 0) & (ei > 0) else f'{i} - {e} = {i - e}'\
                              for i,e,ie,ei in zip(df.b1_wk, df.b2_wk, df.b1_w, df.b2_w)]
    df['dif_l'] = [f'{i} - {e} = {i - e}' for i,e in zip(df.b1_l, df.b2_l)]
    df['dif_lk_perc'] = [f'{round(i/ie, 2)} - {round(e/ei,2)} = {round((i/ie) - (e/ei), 2)}'\
                              if (ie > 0) & (ei > 0) else f'{i} - {e} = {i - e}'\
                              for i,e,ie,ei in zip(df.b1_lk, df.b2_lk, df.b1_l, df.b2_l)]
    df['dif_bet'] = [f'{round(i, 2)} - {round(e, 2)} = {round(i - e, 2)}' for i,e in zip(df.b1_bet, df.b2_bet)]
    df['dif_len_text'] = [f'{i} - {e} = {round(i - e, 2)}' for i,e in zip(df.len_text_boxer1, df.len_text_boxer2)]
    df['modelo'] = str(self.model.model)
    temp = self.confidence_check(df)
    temp = self.select_columns(temp)
    return temp

  def lime_explanation(self, instance):
    explanation = self.explainer.explain_instance(instance, self.model.model.predict_proba, num_features=10)
    dictio = {}
    for feature, weight in explanation.as_list():
      dictio[f'{feature}'] = round(weight, 4)
    return dictio

  def loop_add_explanation(self, x_values):
    lista = []
    for row in x_values:
      res = self.lime_explanation(row)
      lista.append(res)
    return lista

  def make_prediction(self, data_to_predict, id_df):
    '''
    Method to make predictions for the given data.
    Parameters:
      data_to_predict: First dataframe returned from box_data_merger_splitter() function.
    Return:
      temp: Dataframe with predictions from the model, analysis, and explanation of the prediction.
    '''
    df = data_to_predict.copy()
    temp, x_scal= self.predict(df, id_df)
    temp = self.decision_table(temp)
    temp['explanation'] = self.loop_add_explanation(x_scal)
    temp['date_model_pred'] = datetime.now()
    return temp

In [57]:
# predictor = Model_predictions(cluster, scaler, model, explainer)
# trial2 = predictor.make_prediction(trial, new_id_df)

In [58]:
# trial2.head(2)

,initial_index,fight_id,result,endround,endtype,cluster,boxer1_pred,goodpred,prob_win,data_cuant,confidence,boxer1,boxer2,dif_bet,dif_len_text,dif_edad,dif_height,dif_w,dif_wk_perc,dif_l,dif_lk_perc,modelo,b1_id,b2_id,explanation,date_model_pred
0,35,36.0,NaN,NaN,NaN,0,0,,0.475330,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %",jesse rodriguez,sunny edwards,1.08 - 1.14 = -0.06,6005 - 7051 = -1046,2000 - 1996 = 4,163 - 160 = 3,18.0 - 20.0 = -2.0,0.61 - 0.2 = 0.41,0.0 - 0.0 = 0.0,0.0 - 0.0 = 0.0,SVC(probability=True),6502,6532,"{'len_text_boxer2 > -0.14': -0.2289, 'birth_b1...",2023-11-12 23:36:04.708473
1,61,64.0,NaN,NaN,NaN,0,1,,0.935191,"fail 19 % of 1150 (val, test)",,yoanki urrutia,fiodor czerkaszyn,3.1 - 1.36 = 1.74,0 - 0 = 0,1996 - 1990 = 6,188 - 170 = 18,13.0 - 22.0 = -9.0,0.38 - 0.64 = -0.25,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6553,6363,"{'birth_b1 > 0.79': 0.1995, 'b1_wk <= -0.66': ...",2023-11-12 23:36:04.708473
2,62,65.0,NaN,NaN,NaN,0,1,,0.935475,"fail 19 % of 1150 (val, test)",,eduardo nunez,oscar escandon,1.1 - 7.0 = -5.9,13683 - 2008 = 11675,1997 - 1980 = 17,168 - 173 = -5,24.0 - 27.0 = -3.0,1.0 - 0.67 = 0.33,1.0 - 6.0 = -5.0,0.0 - 0.67 = -0.67,SVC(probability=True),6554,2775,"{'birth_b1 > 0.79': 0.1943, 'len_text_boxer1 >...",2023-11-12 23:36:04.708473
3,65,68.0,NaN,NaN,NaN,0,1,,0.912429,"fail 19 % of 1150 (val, test)",,joe cordina,edward vazquez,1.08 - 8.0 = -6.92,6351 - 0 = 6351,1992 - 1995 = -3,175 - 170 = 5,16.0 - 15.0 = 1.0,0.56 - 0.2 = 0.36,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6557,6574,"{'birth_b1 > 0.79': 0.2083, 'b1_wk <= -0.66': ...",2023-11-12 23:36:04.708473
4,66,69.0,NaN,NaN,NaN,0,1,,0.927945,"fail 19 % of 1150 (val, test)",,robeisy ramirez,rafael espinoza,1.07 - 8.0 = -6.93,7087 - 0 = 7087,1994 - 1994 = 0,168 - 185 = -17,13.0 - 21.0 = -8.0,0.62 - 0.86 = -0.24,1.0 - 0.0 = 1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6505,6575,"{'birth_b1 > 0.79': 0.2047, 'b1_wk <= -0.66': ...",2023-11-12 23:36:04.708473


Predictions logger

In [69]:
class CloseException(Exception):
  pass

class Predictions_logger:
  '''
  Class to add model and self predictions to a csv file, in case the file
  doesn't exist it will create it.
  '''
  def __init__(self, file_):
    '''
    Contructor method.
    Parameters:
      file_: filename to record the data, ext: .csv.
    '''
    self.file_ = file_

  def closer(self, message):
      res = input(f'{message}')
      if res == 'c':
        raise CloseException()
      else:
        return res

  def read_file(self):
    '''
    Method for read the file where the model and self predictions are logged.
    Returns:
      df: Dataframe with the mentioned information.
    '''
    try:
      df = pd.read_csv(self.file_).sort_values('fight_id')
      return df
    except KeyError:
      df = pd.read_csv(self.file_)
      return df
    except (FileNotFoundError, pd.errors.EmptyDataError):
      df = pd.DataFrame()
      df.to_csv(self.file_, index=False)
      return df

  def authorization(self, message):
    autho = input(f'{message} (y/n): ')
    if autho == 'y':
      return 1
    else:
      return 0

  def save_file(self, data):
    data.to_csv(self.file_, index=False)
    print('Saved data')

  def display_as_dict(self, data):
    temp = data.to_dict(orient='records')[0]
    keys_to_include = ['cluster', 'boxer1_pred', 'prob_win', 'data_cuant', 'confidence']
    sliced_dict = {key: temp[key] for key in keys_to_include}
    print(sliced_dict)
    keys_to_include = ['boxer1', 'boxer2', 'dif_bet']
    sliced_dict = {key: temp[key] for key in keys_to_include}
    print(sliced_dict)
    keys_to_include = ['dif_len_text', 'dif_edad', 'dif_height']
    sliced_dict = {key: temp[key] for key in keys_to_include}
    print(sliced_dict)
    keys_to_include = ['dif_w', 'dif_wk_perc', 'dif_l', 'dif_lk_perc']
    sliced_dict = {key: temp[key] for key in keys_to_include}
    print(sliced_dict)
    keys_to_include = ['modelo', 'explanation']
    sliced_dict = {key: temp[key] for key in keys_to_include}
    print(sliced_dict)
    print()

  def log_reg(self, data):
    self.display_as_dict(data)
    data['self_pred'] = self.closer('Opinion personal (1 - boxer1 win, 0 - boxer1 loss, c - close): ')
    data['date_self_pred'] = datetime.now()
    df = self.read_file()
    df = pd.concat([df, pd.DataFrame(data)])
    df = df[['fight_id', 'self_pred', 'cluster', 'boxer1_pred', 'prob_win', 'data_cuant', 'confidence',
    'boxer1', 'boxer2', 'dif_bet', 'dif_len_text', 'dif_edad', 'dif_height', 'dif_w', 'dif_wk_perc',
    'dif_l', 'dif_lk_perc', 'modelo', 'b1_id', 'b2_id', 'explanation','date_model_pred','date_self_pred']]
    self.save_file(df)
    print()

  def check_existent_values(self, row, data):
    try:
      id = row.fight_id.values[0]  # Extract the fight_id value from the row
      matching_rows = data[data['fight_id'] == id]  # Find matching rows
      if not matching_rows.empty:  # Check if matching rows exist
        return matching_rows  # Return matching rows
      else:
        return None  # No matching rows found, return 0 or an appropriate value
    except AttributeError:
      return None  # Handle the case where 'row' or 'data' is not a DataFrame
    except KeyError:
      return None

  def record_data_selector(self, data, id='new'):
    if id == 'new':
      try:
        check = self.read_file()
        data = data.drop(data[data.fight_id.isin(check.fight_id.values)].index)
      except AttributeError:
        data = data
    elif id == 'all':
      data = data
    else:
      data = data[data.fight_id == id]
    return data

  def record_data(self, data):
    '''
    Method for record data received from Model_predictions().make_prediction(),
    adding a self prediction.
    Parameters:
      data: Dataframe from the mentioned object in the description.
    '''
    df = data.copy()
    df = self.record_data_selector(df)
    df['prob_win'] = round(df.prob_win, 3)
    df = df[['fight_id', 'cluster', 'boxer1_pred', 'prob_win', 'data_cuant', 'confidence',
        'boxer1', 'boxer2', 'dif_bet', 'dif_len_text', 'dif_edad', 'dif_height',
        'dif_w', 'dif_wk_perc', 'dif_l', 'dif_lk_perc', 'modelo', 'b1_id', 'b2_id',
        'explanation','date_model_pred']]
    size_rows_df = df.shape[0]
    archivo = self.read_file()
    try:
      for i in range(0, size_rows_df):
        temp = df.iloc[i:i+1, :]
        res = self.check_existent_values(temp, archivo)
        if res is not None:
          if res.empty:
            self.log_reg(temp)
          else:
            print('Already exist a record:\n')
            self.display_as_dict(res)
            autho = input("Want to log another one (y/n): ")
            print()
            if autho == 'y':
              self.log_reg(temp)
            else:
              print('siguiente\n')
              continue
        else:
          self.log_reg(temp)
    except CloseException:
        print('Finalizado')

  def delete_row(self, id):
    '''
    Method for delete row.
    Parameters:
      id: Fight id row to delete.
    '''
    try:
      data = pd.read_csv(self.file_)# checar no de problemas el archivo del cual borrar
      filtered_df = data[data['fight_id'] != id]
      delete_row = data[data['fight_id'] == id]
      aut = self.authorization(f'Proced to delete record\n{delete_row.to_dict(orient="records")[0]}\n')
      if aut == 1:
        self.save_file(filtered_df)
        print(f'Row deleted successfully\n')
      else:
        print('Authorization to delete denied\n')
    except Exception as e:
      print(f"Error deleting row: {str(e)} - maybe id mismatch\n")

In [70]:
# logger = Predictions_logger('prueba.csv')

In [72]:
# logger.record_data(trial2)

{'cluster': 0, 'boxer1_pred': 1, 'prob_win': 0.912, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'confidence': ''}
{'boxer1': 'joe cordina', 'boxer2': 'edward vazquez', 'dif_bet': '1.08 - 8.0 = -6.92'}
{'dif_len_text': '6351 - 0 = 6351', 'dif_edad': '1992 - 1995 = -3', 'dif_height': '175 - 170 = 5'}
{'dif_w': '16.0 - 15.0 = 1.0', 'dif_wk_perc': '0.56 - 0.2 = 0.36', 'dif_l': '0.0 - 1.0 = -1.0', 'dif_lk_perc': '0.0 - 0.0 = 0.0'}
{'modelo': 'SVC(probability=True)', 'explanation': {'birth_b1 > 0.79': 0.2083, 'b1_wk <= -0.66': -0.1251, 'b2_w <= -0.73': 0.1101, 'cluster <= 0.12': -0.1074, '-0.42 < len_text_boxer2 <= -0.38': 0.089, '-1.02 < b1_momios_menores <= 0.98': 0.0605, '-1.62 < b1_more_fame <= 0.62': 0.0598, '-0.81 < b1_lk <= -0.38': 0.0552, 'birth_b2 > 0.63': -0.0475, '-0.73 < b2_lk <= -0.26': -0.0406}}

Opinion personal (1 - boxer1 win, 0 - boxer1 loss, c - close): 0
Saved data

{'cluster': 0, 'boxer1_pred': 1, 'prob_win': 0.928, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'conf

In [76]:
# logger.read_file()

In [50]:
# logger.delete_row(64)

Proced to delete record
{'fight_id': 64.0, 'self_pred': 0, 'cluster': 0, 'boxer1_pred': 1, 'prob_win': 0.935, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'confidence': nan, 'boxer1': 'yoanki urrutia', 'boxer2': 'fiodor czerkaszyn', 'dif_bet': '3.1 - 1.36 = 1.74', 'dif_len_text': '0 - 0 = 0', 'dif_edad': '1996 - 1990 = 6', 'dif_height': '188 - 170 = 18', 'dif_w': '13.0 - 22.0 = -9.0', 'dif_wk_perc': '0.38 - 0.64 = -0.25', 'dif_l': '0.0 - 1.0 = -1.0', 'dif_lk_perc': '0.0 - 0.0 = 0.0', 'modelo': 'SVC(probability=True)', 'b1_id': 6553, 'b2_id': 6363, 'explanation': "{'birth_b1 > 0.79': 0.1979, 'b1_wk <= -0.66': -0.1433, 'cluster <= 0.12': -0.0968, '-0.42 < len_text_boxer2 <= -0.38': 0.089, 'b1_more_fame <= -1.62': -0.0611, 'height_b1 > 0.22': 0.056, 'b1_momios_menores <= -1.02': -0.0559, '-0.81 < b1_lk <= -0.38': 0.0549, 'b2_invicto > 0.50': 0.054, 'len_text_boxer1 <= -0.48': -0.0534}"}
 (y/n): y
Saved data
Row deleted successfully



Export predictions

In [80]:
def export_preds(data):
  '''
  Function to export predictions in a csv format.
  Parameters:
    data: Dataframe received from Predictions_logger().read_file(), to export.
  '''
  data = data[['boxer1', 'boxer2', 'self_pred', 'boxer1_pred', 'prob_win', 'data_cuant', 'confidence']]
  current_date = datetime.now().date()
  data.to_csv(f'preds_{current_date}.csv', index=False)

In [81]:
# export_preds(a)